In [1]:
import os
from pathlib import Path
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv, find_dotenv
from datetime import datetime
from IPython.display import display, Markdown
from rich.console import Console

In [2]:
def time_now():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [3]:
try:
    os.environ.clear()
    print(os.environ["OPENAI_API_KEY"][:14])
except KeyError:
    print("❌ OpenAI API cleared.")

❌ OpenAI API cleared.


In [4]:
console = Console()

In [5]:
load_dotenv(find_dotenv(), override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError(
        "❌ OpenAI API key not set. Set OPENAI_API_KEY environment variable."
    )
else:
    console.print(
        f"✅ OpenAI API Key exists and begins [dark_orange]{OPENAI_API_KEY[:14]}...[/]"
    )

✅ OpenAI API Key exists and begins sk-proj-b8HGB3...

In [6]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
message = "Why Langgraph is the best choice for production comapred to CrewAI"

msg = message.replace(" ", "_")


file_path = Path("output/log_search.csv")

if os.path.exists(file_path):
    print(f"✅ File exists: {file_path}")
    with trace("Search"):
        result = await Runner.run(search_agent, message)

        with open("output/log_search.csv", "a") as f:
            # f.write(
            #     f"{gen_trace_id()}|{message}|{result.raw_responses}|{result.new_items}|{result.output_guardrail_results}|{result.last_agent}\n"
            # )
            f.write(f"{time_now()}|{gen_trace_id()}|{message}|{result.last_agent}\n")

        with open(f"output/report_{msg}.md", "w") as f:
            f.write(f"## {message}\n")
            f.write(result.final_output)


else:
    print(f"❌ File does not exist: {file_path}")

✅ File exists: output\log_search.csv


Error getting response: Connection error.. (request_id: None)


APIConnectionError: Connection error.

[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: max retries reached, giving up on this batch.


### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [8]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!


class WebSearchItem(BaseModel):
    reason: str = Field(
        description="Your reasoning for why this search is important to the query."
    )

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(
        description="A list of web searches to perform to best answer the query."
    )


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [9]:
message = "Benefits of Langgraph over CrewAI"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

Error getting response: Connection error.. (request_id: None)


APIConnectionError: Connection error.

In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(
        description="A short 2-3 sentence summary of the findings."
    )

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(
        description="Suggested topics to research further"
    )


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)